<a href="https://colab.research.google.com/github/madhumitha-gv/NLP/blob/main/next_word_prediction_using_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [4]:
!pip install tensorflow

In [5]:
#import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import re

## reading text file

In [6]:
with open("/content/sherlock_output.txt", "r", encoding="utf-8") as file:
    text = file.read()

In [7]:
# Load limited cleaned text to avoid RAM issues
with open("sherlock_output.txt", "r", encoding="utf-8") as f:
    text = f.read().lower()

# Basic cleanup
text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove punctuations
text = re.sub(r'\s+', ' ', text)  # normalize spaces

## Tokenizer

In [8]:
#Tokenizer process
tokenizer = Tokenizer()
#fit
tokenizer.fit_on_texts([text])
#assign length of word index
total_words = len(tokenizer.word_index) + 1

In [9]:
#chek the tokens
tokenizer.word_index

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'in': 5,
 'a': 6,
 'he': 7,
 'that': 8,
 'was': 9,
 'it': 10,
 'his': 11,
 'is': 12,
 'with': 13,
 'as': 14,
 'had': 15,
 'i': 16,
 'for': 17,
 'at': 18,
 'by': 19,
 'on': 20,
 'not': 21,
 'be': 22,
 'from': 23,
 'but': 24,
 'you': 25,
 'or': 26,
 'her': 27,
 'him': 28,
 'which': 29,
 'were': 30,
 'all': 31,
 'this': 32,
 'she': 33,
 'they': 34,
 'are': 35,
 'have': 36,
 'said': 37,
 'an': 38,
 'one': 39,
 'who': 40,
 'their': 41,
 'so': 42,
 'what': 43,
 'when': 44,
 'there': 45,
 'am': 46,
 'been': 47,
 'may': 48,
 'if': 49,
 'no': 50,
 'up': 51,
 'my': 52,
 'them': 53,
 'into': 54,
 'assetsdatacampcomproductionrepositoriesdatasetscabfafdfdsherlocktxt': 55,
 'httpsassetsdatacampcomproductionrepositoriesdatasetscabfafdfdsherlocktxt': 56,
 'its': 57,
 'more': 58,
 'out': 59,
 'would': 60,
 'me': 61,
 'prince': 62,
 'did': 63,
 'only': 64,
 'pierre': 65,
 'we': 66,
 'could': 67,
 'now': 68,
 'has': 69,
 'will': 70,
 'then': 71,
 'some': 72,
 't

In [26]:
lines = text.split('\n')[:10000]

In [27]:
def generate_sequences(text, tokenizer):
    for line in text.split('\n'):
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            yield token_list[:i+1]


In [37]:
input_sequences = []
max_seq_len = 100

for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, min(len(token_list), max_seq_len)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [38]:
#maximum sentence length
max_sequence_len = max([len(seq) for seq in input_sequences])
# input sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [39]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

## onehot encoding

In [40]:
#convert one-hot-encode
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [41]:
max_sequence_len

100

In [44]:
total_words

35522

# model finetuning & architecture

In [42]:
#create model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [45]:
model.build(input_shape=(None, max_sequence_len - 1))
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 99, 100)        │     3,552,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 35522)          │     5,363,822 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,199,868 (103.76 MB)

 Trainable params: 9,066,622 (34.59 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 18,133,246 (69.17 MB)

None


In [46]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#fit the model
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.0927 - loss: 4.1160
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0615 - loss: 4.1035
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.1021 - loss: 3.9653
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0657 - loss: 4.0221
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0896 - loss: 4.0417
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0802 - loss: 4.0332
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0802 - loss: 4.0099
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0771 - loss: 4.0300
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0865 - loss: 4.0572
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0927 - loss: 3.9580
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0771 - loss: 4.0295
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0740 - lo

## inference

In [47]:
#determine a text
seed_text = "I will close the door if"
# predict word number
next_words = 10

for _ in range(next_words):
    #convert to token
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    #path sequences
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #model prediction
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    # get predict words
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [48]:
seed_text

'I will close the door if copyright copyright arthur the copyright copyright arthur the copyright copyright'